[2025-12-11] 독버섯 감지! 
- 필수
   * 교차검증 (GridSearchCV)
   * 데이터 누수 안 됨! => 파이프라인(pipeline 활용)
   * 모델 : 앙상블 계열 => 보팅  /  배깅은 랜덤포레스트 

[1] 모듈로딩 및 데이터 준비 <hr>

In [ ]:
import pandas as pd 
import numpy as np

## ML학습 관련
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

## ML 데이터셋 및 전처리 관련
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, LabelEncoder, OneHotEncoder
from sklearn.ensemble import VotingClassifier

## ML CV, Pipeline 관련 => 모델 일반화/최적 하이퍼파라미터 조사 및 데이터 누수 해결
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline 
from sklearn.compose import ColumnTransformer # 유방암할때 사용해보기 

## ML 성능지표 관련
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
from sklearn.metrics import classification_report 

## 시각화 관련
import matplotlib.pyplot as plt
import graphviz

In [3]:
DATA_FILE = pd.read_csv("../Data/mushrooms.csv")
df = pd.DataFrame(DATA_FILE)
df.head(3)

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m


In [4]:
# # 23개의 컬럼 
# class	            버섯 종류(식용/독성)	edible(e) / poisonous(p)
# cap-shape 	    갓(머리) 모양	        버섯 갓의 전체적인 형태
# cap-surface	    갓 표면 상태	        매끈함, 울퉁불퉁함 등
# cap-color	        갓 색상	                갓의 겉면 색
# bruises	        멍/손상 여부	        눌렀을 때 색 변화가 생기는지
# odor	            냄새	                버섯이 풍기는 향기(독성 식별 핵심)
# gill-attachment	주름살(아가미) 부착 방식	갓에 어떻게 붙어있는지
# gill-spacing	    주름살 간격	촘촘함 / 넓음
# gill-size	        주름살 크기	아가미 길이/두께
# gill-color	    주름살 색상	아가미의 색
# stalk-shape	    줄기 모양	            위/아래로 굵어지거나 가늘어지는 형태
# stalk-root	    줄기 뿌리 형태	        러프/뿌리 모양 등
# stalk-surface-above-ring	고리 위 줄기 표면	고리보다 위쪽의 줄기 표면
# stalk-surface-below-ring	고리 아래 줄기 표면	고리 아래쪽의 줄기 표면
# stalk-color-above-ring	고리 위 줄기 색	고리 위 줄기 부분의 색
# stalk-color-below-ring	고리 아래 줄기 색	고리 아래 줄기 부분의 색
# veil-type	        덮개(베일) 종류	         어린 버섯이 갓·줄기를 덮는 막
# veil-color	    덮개 색	베일의 색
# ring-number	    고리 수	줄기에 있는 고리 개수
# ring-type	        고리 형태	고리 모양(두꺼움/얇음/거침 등)
# spore-print-color	포자 문양 색	버섯을 종이에 놓았을 때 떨어지는 포자색
# population	    개체 분포	주변 개체 수(단독, 소수, 다수 등)
# habitat           서식지      숲, 목초지, 습지 등

# 타겟 컬럼 : class
# 피처 컬럼 : 나머지 22개

In [5]:
print(df.shape)
print(df.info())
print(df.isna().sum()) # 0개 

# 컬럼에 이상한값 없는지 체크하기 
for col in df.columns:
    print(f"\n===== {col} =====")
    print(df[col].value_counts()) # stal-root 컬럼에 ? 존재 2480개


(8124, 23)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   class                     8124 non-null   object
 1   cap-shape                 8124 non-null   object
 2   cap-surface               8124 non-null   object
 3   cap-color                 8124 non-null   object
 4   bruises                   8124 non-null   object
 5   odor                      8124 non-null   object
 6   gill-attachment           8124 non-null   object
 7   gill-spacing              8124 non-null   object
 8   gill-size                 8124 non-null   object
 9   gill-color                8124 non-null   object
 10  stalk-shape               8124 non-null   object
 11  stalk-root                8124 non-null   object
 12  stalk-surface-above-ring  8124 non-null   object
 13  stalk-surface-below-ring  8124 non-null   object
 14  stalk-color-a

In [6]:
# stal-root 컬럼에 ? 결측치 처리하기
df['stalk-root'] = df['stalk-root'].replace('?', "missing")

veil-type -> p 하나 있음 (의미없음)

In [7]:
# 타깃과 피처들의 관계 확인 
# 다른건 뭐 그렇다 치는데, odor 냄새가 구별력이 엄청남 

for col in df.columns:
    if col == "class":
        continue
    ct = pd.crosstab(df[col], df["class"], normalize="index")
    print(f"\n=== {col} vs class ===")
    print(ct)


=== cap-shape vs class ===
class             e         p
cap-shape                    
b          0.893805  0.106195
c          0.000000  1.000000
f          0.506345  0.493655
k          0.275362  0.724638
s          1.000000  0.000000
x          0.532823  0.467177

=== cap-surface vs class ===
class               e         p
cap-surface                    
f            0.672414  0.327586
g            0.000000  1.000000
s            0.447574  0.552426
y            0.463625  0.536375

=== cap-color vs class ===
class             e         p
cap-color                    
b          0.285714  0.714286
c          0.727273  0.272727
e          0.416000  0.584000
g          0.560870  0.439130
n          0.553415  0.446585
p          0.388889  0.611111
r          1.000000  0.000000
u          1.000000  0.000000
w          0.692308  0.307692
y          0.373134  0.626866

=== bruises vs class ===
class           e         p
bruises                    
f        0.306655  0.693345
t        0.8

In [8]:
# 균형인지 불균형인지 확인하기
df["class"].value_counts(normalize=True)

class
e    0.517971
p    0.482029
Name: proportion, dtype: float64

In [9]:
# 필요없는 컬럼 제거 
df = df.drop(columns=["veil-type","gill-attachment","ring-number"])

print(df.columns)
print(df.shape)
# veil-type은 하나만 있고
# gill/ ring_number는 데이터 편향이 너무심함 하나의 값에 90%이상 데이터가 있는데
# 그럼 학습해도 의미가 없을 것 같음

Index(['class', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
       'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape', 'stalk-root',
       'stalk-surface-above-ring', 'stalk-surface-below-ring',
       'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-color',
       'ring-type', 'spore-print-color', 'population', 'habitat'],
      dtype='object')
(8124, 20)


In [10]:
# [3] 타깃 피쳐 분리
df["class"] = df["class"].map({"e":0, "p":1}) # 타깃 컬럼 0과 1로 

featureDF = df.drop(columns=["class"])
targetSR = df["class"]

print(featureDF.shape, targetSR.shape)

(8124, 19) (8124,)


In [11]:
# [4] 학습용/ 테스트용 데이터 분리
x_train, x_test, y_train, y_test = train_test_split(featureDF,
                                                    targetSR,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=targetSR
                                                    )

print("[Train]", x_train.shape, y_train.shape)
print("[Test]", x_test.shape, y_test.shape)

[Train] (6499, 19) (6499,)
[Test] (1625, 19) (1625,)


### RandomForest 활용해보기 

In [ ]:
#[5] OneHotEncoder / RandomForest 파이프라인 
ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)

rf_clf = RandomForestClassifier(random_state=42)

rf_pipe = Pipeline([
    ("ohe", ohe),
    ("clf", rf_clf)
])


In [ ]:
#[6] 랜덤 포레스트 + 교차검증 (GridSearchCV 활용) / 하이퍼파라미터 튜닝도 진행 
rf_param_grid = {
    "clf__n_estimators": [100, 200, 300],
    "clf__max_depth": [10,20,30],
    "clf__min_samples_split": [2, 5],
    "clf__min_samples_leaf": [1, 2]
}
            
scoring = {
    "accuracy": "accuracy",
    "f1_micro": "f1_micro",
    "precision_micro": "precision_micro",
    "recall_micro": "recall_micro"
}

rf_grid = GridSearchCV(rf_pipe,
                       param_grid=rf_param_grid,
                       cv=5,              # 5겹 교차검증
                       scoring=scoring,      # 오늘 수업시간에 했떤거 적용해보기
                       verbose=1,
                       refit='f1_micro'
)

rf_grid.fit(x_train, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


,estimator,Pipeline(step...m_state=42))])
,param_grid,"{'clf__max_depth': [10, 20, ...], 'clf__min_samples_leaf': [1, 2], 'clf__min_samples_split': [2, 5], 'clf__n_estimators': [100, 200, ...]}"
,scoring,"{'accuracy': 'accuracy', 'f1_micro': 'f1_micro', 'precision_micro': 'precision_micro', 'recall_micro': 'recall_micro'}"
,n_jobs,None
,refit,'f1_micro'
,cv,5
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,categories,'auto'


In [17]:
# [6-1] 학습 결과 확인 
print(f" best_params_:, {rf_grid.best_params_}")
print(f" best_score_ :, {rf_grid.best_score_}")
print(f" best_score_ :, {rf_grid.best_estimator_}")

 best_params_:, {'clf__max_depth': 10, 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 2, 'clf__n_estimators': 100}
 best_score_ :, 1.0
 best_score_ :, Pipeline(steps=[('ohe',
                 OneHotEncoder(handle_unknown='ignore', sparse_output=False)),
                ('clf', RandomForestClassifier(max_depth=10, random_state=42))])


In [31]:
# [7] 테스트 해보깅 
rf_best = rf_grid.best_estimator_

y_pred_rf = rf_best.predict(x_test)

print(" RandomForest- Test Confusion Matrix ")
print(confusion_matrix(y_test, y_pred_rf))

print("\n RandomForest - Test Classification Report ")
print(classification_report(y_test, y_pred_rf))

# 1.0이 나왓다고 해서 이상해서 계속 찾아보고 했는데
# mushroom 데이터에서 아까 odor과 같은 냄새에서 분리를 엄청 잘하는것도 있고
# 이외에도 특정 컬럼에서 식용과 독인걸 확실하게 구분할 수 있는 것들이 있어서
# 1.0이 비정상은 아닌 것 같음


 RandomForest- Test Confusion Matrix 
[[842   0]
 [  0 783]]

 RandomForest - Test Classification Report 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       842
           1       1.00      1.00      1.00       783

    accuracy                           1.00      1625
   macro avg       1.00      1.00      1.00      1625
weighted avg       1.00      1.00      1.00      1625



### Voting 방식 해보기 
- (RandomForest는 앞에서 써봣으니까 빼고 배운것중에 KNN이랑 DecisionTree)

In [21]:
# [8] Voting 구성 ( knn + decision )  / 파이프라인 구성 

# 모델 각각 인스턴스 생성 
knn = KNeighborsClassifier()
dt = DecisionTreeClassifier(random_state=42)

# 보팅 모델 인스턴스 생성
voting_clf = VotingClassifier(estimators=[("knn", knn),("dt", dt)])

# 파이프라인 설계 
voting_pipe = Pipeline([
    ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
    ("clf", voting_clf)
])



In [28]:
# [9]GridSearchCV 설정 
voting_param_grid = {"clf__knn__n_neighbors": [3, 5, 7],
                     "clf__dt__max_depth": [None,5,10,15,20],
                     "clf__dt__min_samples_split": [2, 5],}

scoring = {"accuracy": "accuracy",
           "f1_micro": "f1_micro",
           "precision_micro": "precision_micro",
           "recall_micro": "recall_micro"}


voting_grid = GridSearchCV(voting_pipe,
                           param_grid=voting_param_grid,
                           cv=5,
                           scoring=scoring,
                           refit="f1_micro",
                           verbose=1)

voting_grid.fit(x_train, y_train)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


,estimator,Pipeline(step...tate=42))]))])
,param_grid,"{'clf__dt__max_depth': [None, 5, ...], 'clf__dt__min_samples_split': [2, 5], 'clf__knn__n_neighbors': [3, 5, ...]}"
,scoring,"{'accuracy': 'accuracy', 'f1_micro': 'f1_micro', 'precision_micro': 'precision_micro', 'recall_micro': 'recall_micro'}"
,n_jobs,None
,refit,'f1_micro'
,cv,5
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,categories,'auto'


In [29]:
# [9-1] 학습 결과 확인 
print(f" best_params_:, {voting_grid.best_params_}")
print(f" best_score_ :, {voting_grid.best_score_}")
print(f" best_score_ :, {voting_grid.best_estimator_}")

 best_params_:, {'clf__dt__max_depth': None, 'clf__dt__min_samples_split': 2, 'clf__knn__n_neighbors': 3}
 best_score_ :, 0.9995384615384616
 best_score_ :, Pipeline(steps=[('ohe',
                 OneHotEncoder(handle_unknown='ignore', sparse_output=False)),
                ('clf',
                 VotingClassifier(estimators=[('knn',
                                               KNeighborsClassifier(n_neighbors=3)),
                                              ('dt',
                                               DecisionTreeClassifier(random_state=42))]))])


In [34]:
#[10] 테스트 데이터 해보기
best_voting = voting_grid.best_estimator_

y_pred = best_voting.predict(x_test)

print("\n Confusion Matrix ")
print(confusion_matrix(y_test, y_pred))

print("\n Classification Report ")
print(classification_report(y_test, y_pred))


 Confusion Matrix 
[[842   0]
 [  0 783]]

 Classification Report 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       842
           1       1.00      1.00      1.00       783

    accuracy                           1.00      1625
   macro avg       1.00      1.00      1.00      1625
weighted avg       1.00      1.00      1.00      1625



In [ ]:
# 놓친부분?? 
# voting할때 2개 넣으면 투표가 되는지 ?
# 최적의 하이퍼파라미터 찾기에서 gini, log_loss 안보고 defalut값 써버림 